# Keeping Trump on Topic: LIN353C Final Project

By Hannah Brinsko and Aditya Kharosekar

In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import datetime
import gensim
from gensim.models.keyedvectors import KeyedVectors

C:\Users\Aditya Kharosekar\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Preprocessing - Scraping tweets, and cleaning them up

Importing tweets from the CSV file - 

In [2]:
tweets_csv = pd.read_csv("tweets.csv")
trump_tweets = tweets_csv[tweets_csv['handle']=="realDonaldTrump"]
print(trump_tweets.shape)
print(trump_tweets['time'].head())

(3218, 28)
5     2016-09-27T22:13:24
8     2016-09-27T21:08:22
11    2016-09-27T20:31:14
12    2016-09-27T20:14:33
13    2016-09-27T20:06:25
Name: time, dtype: object


Looking at the CSV file, we see that it contains tweets only up to 09/27/2016. We need his more recent tweets as well

## Getting his most recent ~3200 tweets.

3200 is approximately the limit to how many tweets Tweepy allows us to scrape. As it turns out, this is more than enough for our use when combined with our CSV.

In [3]:
import tweepy
import json
from tweepy import OAuthHandler
import codecs

consumer_key = "i387QW7Eqgh12UHmK3VoQO9K5"
consumer_secret = "BQI8c5eKale4etdA21mawnFqOmAziDQpnThm679V7UtLjbWlMG"
access_token = "816857419338764288-S8Ay111O2Mo32QAs88tSnv5uKvmGCkF"
access_secret = "HVU19yLuV0klltJl1fsDibAi7Hiq1U4GwsEV9kozTAc1m"

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

all_tweets = []

new_tweets = api.user_timeline(screen_name="realDonaldTrump", count=200)


all_tweets.extend(new_tweets)
oldest = all_tweets[-1].id-1

t = new_tweets[0];

while len(new_tweets) > 0:
   new_tweets = api.user_timeline(screen_name = "realDonaldTrump", count=200, max_id = oldest)
   all_tweets.extend(new_tweets)
   oldest = all_tweets[-1].id-1;

In [4]:
print(len(all_tweets))

3246


We now have his most recent tweets.

However, there is significant overlap between the tweets that we have scraped from his account and the tweets that are in the CSV file.

The latest tweet in the CSV file was posted on September 27, 2016 at 22:13:24. So, we need to keep any scraped tweets which were posted after this.

In [5]:
tweets = []
i = 0
while (all_tweets[i].created_at!=datetime.datetime(2016, 9, 27, 22, 13, 24)):
    tweets.append(all_tweets[i].text)
    i+=1

In [6]:
tweets1 = trump_tweets['text']
tweets1 = tweets1.tolist()
for t in tweets1:
    tweets.append(t)
print(type(tweets))
print("We have ", len(tweets), "tweets to work with")

<class 'list'>
We have  4645 tweets to work with


Making distributional vectors from each tweet

But to do that, we need to - 
1. Remove any twitter links and image links
2. Remove any stopwords
3. Make sure that we have a list of tweets where each tweet is a string
4. Then use CountVectorizer http://scikit-learn.org/stable/modules/feature_extraction.html#common-vectorizer-usage

### Removing links

In [7]:
temp_tweets = []
for t in tweets:
    temp_tweets.append(t.lower().split())

print(temp_tweets[1])
for t in temp_tweets:
    for w in t:
        if "http" in w or "@" in w: #I've removed any instances where he tags anyone in his tweets. 
                                    #I thought the word vectors might be too sparse if I left those in.
            t.remove(w)
print(temp_tweets[1]) 

['congratulations', 'to', 'justice', 'neil', 'gorsuch', 'on', 'his', 'elevation', 'to', 'the', 'united', 'states', 'supreme', 'court.', 'a', 'great', 'day', 'for', 'americ…', 'https://t.co/rm9lftaeps']
['congratulations', 'to', 'justice', 'neil', 'gorsuch', 'on', 'his', 'elevation', 'to', 'the', 'united', 'states', 'supreme', 'court.', 'a', 'great', 'day', 'for', 'americ…']


## NOTE: This link removal is not working properly. Have to fix later

### Removing stopwords

In [8]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

for t in temp_tweets:
    for w in t:
        if w in stop:
            t.remove(w)

Succesfully removed stopwords. At this point, each tweet is a list of words and temp_tweets is a list. What we need to use CountVectorizer is a list where each element is a string.

Therefore, we need to convert each tweet from a lists of words to a string.

In [9]:
tweets = []
for t in temp_tweets:
    tweets.append(' '.join(t))
type(tweets[0])

str

## Our methodology for classifying tweets

Step 0 - Download a pre-trained Word2Vec model. We tried training our own model, but we did not have enough data.

Step 1 - Hand tag some number of tweets (we ended up tagging about 280 tweets) and classify them into the following categories - 
1. Foreign Policy / International News
2. Domestic Policy / domestic news
3. Tweets about the media
4. Attack tweets
5. Other tweets
6. Tweets about the election

Step 2 - From our hand-tagged corpus, and for each category, create a list of words used.

Step 3 - Create a word vector for each category by summing up the individual word vectors

Step 4 - For each subsequent tweet, find cosine similarity between it and each category vector. Assign that tweet to the category it is most similar to

### Step 0 - Downloading a pre-trained Word2Vec model

In [13]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r', encoding = 'utf8')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print("Done.",len(model)," words loaded")
    return model

glove_model = loadGloveModel('glove.txt')

Loading Glove Model
Done. 400000  words loaded


In [ ]:
glove_model['hillary']

In [10]:
google_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [18]:
google_model.similarity('King', 'Kong')

0.31701805595455718

google_model is our pre-trained model which we will be using.

### Step 1 - Hand tagging tweets

In [37]:
tagged_tweets = pd.read_csv("tagged_tweets.csv")
tagged_tweets.columns = ['Tweet', 'Category']
tweet_text = tagged_tweets['Tweet']
tweet_categories = tagged_tweets['Category']

In [ ]:
foreign = []
domestic =[]
media =[]
attack = []
election = []
other = []



In [35]:
type(a[40])

str